In [12]:
%pip install langchain_community openai elasticsearch tiktoken langchain_elasticsearch


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [11]:
from langchain_community.chat_models import ChatOpenAI
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import ElasticsearchStore
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
import warnings
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from elasticsearch import Elasticsearch
import os
import ssl 

warnings.filterwarnings("ignore", category=DeprecationWarning)

docs_path = os.path.join(os.path.dirname(os.path.abspath('')), 'docs')

ES_HOST = "https://localhost:9200"
ES_USER = "elastic"
ES_PASSWORD = "DdgnuaPpFlewuaq_R7BD"
ES_API_KEY = "RDlZV0VwRUI3VTZhdGRoOXo4ZUk6N3RIMVFCNDJTSW1SMjQ3UXlfbmFVUQ=="
CA_CERT_PATH = "/Users/jeongmin/http_ca.crt"  # CA 인증서 경로

api_key_header = "ApiKey " + ES_API_KEY
headers={"Authorization": api_key_header, "Content-Type": "application/json"}

llm = ChatOpenAI(model_name="gpt-4o", temperature=0.1)

# Initialize the embedding model
embeddings = OpenAIEmbeddings()

# Create an SSL context
ssl_context = ssl.create_default_context(cafile=CA_CERT_PATH)
ssl_context.check_hostname = False
ssl_context.verify_mode = ssl.CERT_NONE

# Create Elasticsearch client
es_client = Elasticsearch(
    [f"https://localhost:9200"],
    basic_auth=(ES_USER, ES_PASSWORD),
    ssl_context=ssl_context
)

loader = TextLoader(f"{docs_path}/few-shot-prompting.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=0)
docs = text_splitter.split_documents(documents)


# Initialize the Elasticsearch vector store
es_store = ElasticsearchStore.from_documents(
    documents=docs,
    es_connection=es_client, 
    index_name="rag_chatbot_index",
    embedding=embeddings
)

Created a chunk of size 604, which is longer than the specified 500
Created a chunk of size 506, which is longer than the specified 500
Created a chunk of size 537, which is longer than the specified 500
Created a chunk of size 501, which is longer than the specified 500


In [29]:
from elasticsearch import Elasticsearch

API_KEY = "V0h1dUVwRUJPcWptOVdZRnFucVM6TUFmOHNNUXZSV0stS3VSQng4RU1UZw=="

client = Elasticsearch(
  "https://cfb5e36d38c24a80850878244f93c284.us-central1.gcp.cloud.es.io:443",
  api_key=API_KEY
)


documents = [
  { "index": { "_index": "chatbot-rag", "_id": "9780553351927"}},
  {"name": "Snow Crash", "author": "Neal Stephenson", "release_date": "1992-06-01", "page_count": 470, "_extract_binary_content": True, "_reduce_whitespace": True, "_run_ml_inference": True},
  { "index": { "_index": "chatbot-rag", "_id": "9780441017225"}},
  {"name": "Revelation Space", "author": "Alastair Reynolds", "release_date": "2000-03-15", "page_count": 585, "_extract_binary_content": True, "_reduce_whitespace": True, "_run_ml_inference": True},
  { "index": { "_index": "chatbot-rag", "_id": "9780451524935"}},
  {"name": "1984", "author": "George Orwell", "release_date": "1985-06-01", "page_count": 328, "_extract_binary_content": True, "_reduce_whitespace": True, "_run_ml_inference": True},
  { "index": { "_index": "chatbot-rag", "_id": "9781451673319"}},
  {"name": "Fahrenheit 451", "author": "Ray Bradbury", "release_date": "1953-10-15", "page_count": 227, "_extract_binary_content": True, "_reduce_whitespace": True, "_run_ml_inference": True},
  { "index": { "_index": "chatbot-rag", "_id": "9780060850524"}},
  {"name": "Brave New World", "author": "Aldous Huxley", "release_date": "1932-06-01", "page_count": 268, "_extract_binary_content": True, "_reduce_whitespace": True, "_run_ml_inference": True},
  { "index": { "_index": "chatbot-rag", "_id": "9780385490818"}},
  {"name": "The Handmaid's Tale", "author": "Margaret Atwood", "release_date": "1985-06-01", "page_count": 311, "_extract_binary_content": True, "_reduce_whitespace": True, "_run_ml_inference": True},
]

client.bulk(operations=documents, pipeline="ent-search-generic-ingestion")


ObjectApiResponse({'errors': False, 'took': 58, 'ingest_took': 1, 'items': [{'index': {'_index': 'chatbot-rag', '_id': '9780553351927', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 0, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'chatbot-rag', '_id': '9780441017225', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 1, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'chatbot-rag', '_id': '9780451524935', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 2, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'chatbot-rag', '_id': '9781451673319', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 0, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'chatbot-rag', '_id': '9780060850524', '_version': 1, 'result': 'created', '_shards': {'total':

In [30]:
client.search(index="chatbot-rag", q="snow")

ObjectApiResponse({'took': 29, 'timed_out': False, '_shards': {'total': 2, 'successful': 2, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 1, 'relation': 'eq'}, 'max_score': 1.5169399, 'hits': [{'_index': 'chatbot-rag', '_id': '9780553351927', '_score': 1.5169399, '_source': {'release_date': '1992-06-01', 'author': 'Neal Stephenson', 'name': 'Snow Crash', 'page_count': 470}}]}})

In [31]:
from langchain_elasticsearch import DenseVectorStrategy
from langchain_elasticsearch import ElasticsearchStore

CLOUD_ID = "35e14e0f2e12492e832ab6a800d0b2c1:dXMtY2VudHJhbDEuZ2NwLmNsb3VkLmVzLmlvJGNmYjVlMzZkMzhjMjRhODA4NTA4NzgyNDRmOTNjMjg0JGYyNDFiNzhiMDY5MjQ4NzFhY2RkZmFiOWMyMzg5MTdl"
API_KEY = "V0h1dUVwRUJPcWptOVdZRnFucVM6TUFmOHNNUXZSV0stS3VSQng4RU1UZw=="

index = "chatbot-rag"
client = Elasticsearch(
  "https://cfb5e36d38c24a80850878244f93c284.us-central1.gcp.cloud.es.io:443",
  api_key=API_KEY
)

# Initialize the Elasticsearch vector store
es_store = ElasticsearchStore.from_documents(
    documents=docs,
    es_connection=client, 
    index_name=index,
    embedding=embeddings, 
    strategy=DenseVectorStrategy(hybrid=True)
)

q = "what is few-shot prompting?"

es_store.similarity_search(q, k=3)


[Document(metadata={'source': '/Users/jeongmin/PycharmProjects/rag-with-elasticsearch/chatbot-rag/docs/few-shot-prompting.txt'}, page_content="Overall, it seems that providing examples is useful for solving some tasks. When zero-shot prompting and few-shot prompting are not sufficient, it might mean that whatever was learned by the model isn't enough to do well at the task. From here it is recommended to start thinking about fine-tuning your models or experimenting with more advanced prompting techniques. Up next we talk about one of the popular prompting techniques called chain-of-thought prompting which has gained a lot of popularity."),
 Document(metadata={'source': '/Users/jeongmin/PycharmProjects/rag-with-elasticsearch/chatbot-rag/docs/few-shot-prompting.txt'}, page_content="According to Touvron et al. 2023 few shot properties first appeared when models were scaled to a sufficient size (Kaplan et al., 2020).\n\nLet's demonstrate few-shot prompting via an example that was presented

In [33]:
from langchain_elasticsearch import SparseVectorStrategy

# Initialize the Elasticsearch vector store
db = ElasticsearchStore.from_documents(
    documents=docs,
    es_connection=client, 
    index_name=index,
    embedding=embeddings, 
    strategy=SparseVectorStrategy(model_id=".elser_model_2")
)

db.client.indices.refresh(index="chatbot-rag")


results = db.similarity_search(
    "What is few shot prompting", k=4
)
print(results[0])


ValueError: Cannot do sparse retrieval with a query_vector. Inference is currently always applied in Elasticsearch.